In [9]:
# ✅ Required Libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, HTML
import re

# ✅ Step 1: Real YouTube Data (Updated & Working)
data = {
    'video_id': [1, 2, 3, 4, 5],
    'title': [
        'Python Tutorial for Beginners',
        'Telugu Comedy Movie Clips',
        'Machine Learning Full Course',
        'Funny Telugu Short Film',
        'Deep Learning Explained in Telugu'
    ],
    'description': [
        'Learn Python from scratch with hands-on examples.',
        'Non-stop Brahmanandam comedy scenes for laughter.',
        'Start your ML journey with this full-length course by Simplilearn.',
        'A hilarious short film in Telugu about hostel life.',
        'Complete Deep Learning course explained in Telugu with examples.'
    ],
    'duration': ['4:22:10', '15:34', '3:45:00', '18:45', '1:25:18'],
    'views': ['8M views', '5M views', '3.2M views', '2M views', '1.1M views'],
    'uploaded': ['2 years ago', '1 year ago', '2 years ago', '3 months ago', '8 months ago'],
    'thumbnail': [
        'https://img.youtube.com/vi/eWRfhZUzrAc/0.jpg',      # Python
        'https://img.youtube.com/vi/mC5coI6a_9s/0.jpg',       # Comedy
        'https://img.youtube.com/vi/i_LwzRVP7bg/0.jpg',       # ML Course
        'https://img.youtube.com/vi/MX8xxc7CZkQ/0.jpg',       # Short film
        'https://img.youtube.com/vi/aircAruvnKk/0.jpg'        # Deep Learning
    ],
    'video_url': [
        'https://www.youtube.com/watch?v=eWRfhZUzrAc',
        'https://www.youtube.com/watch?v=mC5coI6a_9s',
        'https://www.youtube.com/watch?v=i_LwzRVP7bg',
        'https://www.youtube.com/watch?v=MX8xxc7CZkQ',
        'https://www.youtube.com/watch?v=aircAruvnKk'
    ]
}

# ✅ Step 2: Preprocessing
df = pd.DataFrame(data)
df['content'] = df['title'] + " " + df['description']
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['content'])
similarity_matrix = cosine_similarity(tfidf_matrix)

# ✅ Step 3: CSS Style for Cards
style = """
<style>
.container {
    display: flex;
    flex-wrap: wrap;
    gap: 20px;
    justify-content: flex-start;
}
.card {
    width: 240px;
    background: #fff;
    border-radius: 12px;
    box-shadow: 0 4px 10px rgba(0,0,0,0.1);
    overflow: hidden;
    transition: transform 0.2s;
    position: relative;
}
.card:hover {
    transform: scale(1.02);
}
.card img {
    width: 100%;
    height: 130px;
    object-fit: cover;
}
.duration {
    position: absolute;
    bottom: 8px;
    right: 8px;
    background: rgba(0,0,0,0.75);
    color: #fff;
    padding: 2px 6px;
    font-size: 12px;
    border-radius: 3px;
}
.card h4 {
    font-size: 14px;
    margin: 8px;
}
.card p {
    font-size: 12px;
    color: #444;
    margin: 6px 8px;
}
.card .meta {
    font-size: 11px;
    color: gray;
    margin: 0 8px 8px;
}
.progress {
    background-color: #eee;
    border-radius: 5px;
    overflow: hidden;
    height: 6px;
    margin: 4px 8px;
}
.progress-bar {
    height: 100%;
    background: linear-gradient(to right, #2196F3, #21CBF3);
    transition: width 0.4s ease;
}
h4 span {
    background: yellow;
}
</style>
"""

# ✅ Step 4: Highlight Matching Text
def highlight_text(text, keyword):
    return re.sub(f"({re.escape(keyword)})", r"<span>\1</span>", text, flags=re.I)

# ✅ Step 5: Recommend Function
def recommend(video_title, df, similarity_matrix):
    video_title = video_title.strip().lower()
    matches = df[df['title'].str.lower().str.contains(video_title)]

    if matches.empty:
        return HTML("<h3 style='color:red;'>❌ Video not found. Try a different keyword.</h3>")

    idx = matches.index[0]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:4]
    video_indices = [i[0] for i in sim_scores]

    # Matched video block
    matched = df.loc[idx]
    matched_html = f"""
    <h2 style='color:#4CAF50;'>📌 Matched Video</h2>
    <div style='display:flex;gap:20px;align-items:center;margin-bottom:30px;'>
        <a href="{matched['video_url']}" target="_blank">
            <img src="{matched['thumbnail']}" width="180" style="border-radius:10px;">
        </a>
        <div>
            <h3 style="margin:0;">{highlight_text(matched['title'], video_title)}</h3>
            <p style="margin:5px 0;color:gray;">{matched['description']}</p>
            <p style="font-size:12px;color:#555;">{matched['views']} • {matched['uploaded']} • ⏱ {matched['duration']}</p>
        </div>
    </div>
    """

    # Recommended cards
    cards = ""
    for i in video_indices:
        score = similarity_matrix[idx][i]
        cards += f"""
        <div class="card" title="Click to watch">
            <a href="{df['video_url'][i]}" target="_blank">
                <img src="{df['thumbnail'][i]}" alt="thumbnail">
                <div class="duration">{df['duration'][i]}</div>
            </a>
            <h4>{highlight_text(df['title'][i], video_title)}</h4>
            <p>{df['description'][i]}</p>
            <div class="progress">
                <div class="progress-bar" style="width:{score*100:.0f}%"></div>
            </div>
            <div class="meta">{df['views'][i]} • {df['uploaded'][i]}</div>
        </div>
        """

    recommended_html = f"""
    <h2 style='color:#2196F3;'>🎬 Recommended Videos</h2>
    <div class='container'>{cards}</div>
    """
    return HTML(style + matched_html + recommended_html)

# ✅ Step 6: Input from User
user_input = input("🔍 Enter a keyword or part of a video title: ")
display(recommend(user_input, df, similarity_matrix))


🔍 Enter a keyword or part of a video title: python
